<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/04_CAG_GC.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip3 install -qU langchain-upstage

In [2]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()
context = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""

In [6]:
chain.invoke({"question": "What is the license under which SOLAR 10.7B is available?", "Context": context})

'The license under which SOLAR 10.7B is available is the Apache 2.0 license.'

In [7]:
chain.invoke({"question": "Who is the created SOLAR 10.7B?", "Context": context})

'The information is not present in the context.'

In [8]:
chain.invoke({"question": "Did Google provide resources for the SOLAR 10.7B project?", "Context": context})

'The information is not present in the context.'

In [9]:
chain.invoke({"question": "Why DUS is good", "Context": context})

'The information is not present in the context.'

In [10]:
answer = chain.invoke({"question": "What is the name of the variant fine-tuned for instruction-following capabilities?", "Context": context})
print(answer)

The variant fine-tuned for instruction-following capabilities is called SOLAR 10.7B-Instruct.


## Groundedness Check with LangChain and Upstage
![Groundedness](./figures/gc.png)


In [11]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = chain.invoke(
    {
        "question": "What is DUS?",
        "Context": context,
    }
)
print("Potential answer: ", answer)
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print("GC check result: ", gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

Potential answer:  DUS stands for depth up-scaling, a method for scaling large language models (LLMs) proposed in the context. It encompasses depthwise scaling and continued pretraining, and is described as simple yet effective in scaling up high-performance LLMs from small ones.
GC check result:  grounded
✅ Groundedness check passed


In [12]:
# GC
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

answer = "Solar 10.7B is available to the public with a non-commercial license."
gc_result = groundedness_check.invoke({"context": context, "answer": answer})

print(gc_result)
if gc_result.lower().startswith("grounded"):
    print("✅ Groundedness check passed")
else:
    print("❌ Groundedness check failed")

notGrounded
❌ Groundedness check failed


# Excercise 

Write code to check GC and make LLM more secure. For example, check GC several times until LLM gives you a reliable answer.